In [1]:
!pip install -q ipympl && pip install plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.1 MB/s eta 0:00:00


In [2]:
!pip install  -q -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.1 MB/s eta 0:00:00


In [3]:
import matplotlib.pyplot as plt  # Used for creating static, interactive, and animated visualizations in Python
import numpy as np  # Fundamental package for scientific computing with Python; provides support for arrays, matrices, and high-level math functions
import pandas as pd  # Data manipulation and analysis library, providing data structures and operations for manipulating numerical tables and time series
import plotly.graph_objs as go  # Graph Objects from Plotly, used for building complex interactive visualizations
from scipy.stats import norm, multivariate_normal  # Provides objects for the normal and multivariate normal distributions, used in probability and statistics
from sklearn import preprocessing  # Provides utilities for scaling, transforming, and wrangling data
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay  # Functions to compute and visualize a confusion matrix to evaluate the accuracy of a classification
from sklearn.decomposition import PCA  # Principal Component Analysis (PCA) algorithm from scikit-learn used for dimensionality reduction
import logging  # Logging library to log messages and track events within software; useful for debugging
from plotly.io import write_image
np.random.seed(47)

In [4]:
# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def generate_samples_from_distribution(N, distribution_params):
    """
    Generates samples from a Gaussian Mixture Model based on provided distribution parameters.

    Parameters:
    N (int): Number of samples to generate.
    distribution_params (dict): Dictionary containing 'mu', 'Sigma', and 'priors' which define
                                the means, covariance matrices, and mixing proportions for each Gaussian component.

    Returns:
    tuple: Tuple containing:
        - X (numpy.ndarray): Array of shape (N, n) containing the generated samples.
        - labels (numpy.ndarray): Array of shape (N,) containing the class label for each sample.

    Raises:
    ValueError: If `N` is not positive or distribution parameters are inconsistent.
    """

    if N <= 0:
        logging.error("Number of samples, N, must be a positive integer.")
        raise ValueError("Number of samples, N, must be a positive integer.")
    if not all(k in distribution_params for k in ['mu', 'Sigma', 'priors']):
        logging.error("Distribution parameters must include 'mu', 'Sigma', and 'priors'.")
        raise ValueError("Distribution parameters must include 'mu', 'Sigma', and 'priors'.")

    try:
        n = distribution_params['mu'].shape[1]
        X = np.zeros([N, n])
        labels = np.zeros(N)
        u = np.random.rand(N)
        thresholds = np.cumsum(distribution_params['priors'])
        thresholds = np.insert(thresholds, 0, 0)

        L = np.array(range(1, len(distribution_params['priors']) + 1))
        for l in L:
            indices = np.argwhere((thresholds[l-1] <= u) & (u <= thresholds[l]))[:, 0]
            labels[indices] = l * np.ones(len(indices)) - 1
            if n == 1:
                X[indices, 0] = norm.rvs(distribution_params['mu'][l-1], distribution_params['Sigma'][l-1], len(indices))
            else:
                X[indices, :] = multivariate_normal.rvs(distribution_params['mu'][l-1], distribution_params['Sigma'][l-1], len(indices))
        logging.info("Sample generation successful.")
        return X, labels
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        raise

In [5]:
# Define the number of samples to generate
total_samples = 10000

# Initialize Gaussian Mixture Model (GMM) parameters
gmm_parameters = {
    'priors': np.array([0.7, 0.3]),  # Class priors for each of the two classes
    'mu': np.array([[-1, 1, -1, 1],  # Mean vectors for each class
                    [1, 1, 1, 1]]),
    'Sigma': np.array([[[2, -0.5, 0.3, 0],  # Covariance matrices for each class
                        [-0.5, 1, -0.5, 0],
                        [0.3, -0.5, 1, 0],
                        [0, 0, 0, 2]],
                       [[1, 0.3, -0.2, 0],
                        [0.3, 2, 0.3, 0],
                        [-0.2, 0.3, 1, 0],
                        [0, 0, 0, 3]]])
}

# Example loss matrix
# L(D=i | Y=j)
# L(D=0|Y=1) is the cost of predicting class 0 when true class is 1
# L(D=1|Y=0) is the cost of predicting class 1 when true class is 0
loss_matrix = np.array([[0, 1],   # Costs when true class is 0
                        [1, 0]])  # Costs when true class is 1

gamma = (loss_matrix[1, 0] * gmm_parameters['priors'][0]) / (loss_matrix[0, 1] * gmm_parameters['priors'][1])

# Generate the samples and labels using the defined GMM parameters
data, class_labels = generate_samples_from_distribution(total_samples, gmm_parameters)

# Calculate the number of samples in each class
number_of_classes = len(gmm_parameters['priors'])  # Determine the number of classes from the length of priors array
dimensionality = data.shape[1]  # Get the number of dimensions from the data array shape
class_distribution = np.array([np.sum(class_labels == i) for i in range(number_of_classes)])  # Count samples in each class

# Output the distribution of samples across the classes
print(f"Class 0 has {class_distribution[0]} samples.")  # Print the number of samples in Class 0
print(f"Class 1 has {class_distribution[1]} samples.")  # Print the number of samples in Class 1

Class 0 has 6997 samples.
Class 1 has 3003 samples.


In [6]:
# Create a 3D scatter plot using Plotly
fig = go.Figure(data=[
    # Scatter plot for Class 0
    go.Scatter3d(
        x=data[class_labels == 0, 0],  # X-coordinates of Class 0
        y=data[class_labels == 0, 1],  # Y-coordinates of Class 0
        z=data[class_labels == 0, 2],  # Z-coordinates of Class 0
        mode='markers',  # Use markers to represent points
        marker=dict(
            size=5,  # Size of the markers
            color='yellow',  # Color of markers for Class 0
            opacity=0.8  # Marker opacity
        ),
        name='Class 0 (Yellow)'  # Label for legend
    ),
    # Scatter plot for Class 1
    go.Scatter3d(
        x=data[class_labels == 1, 0],  # X-coordinates of Class 1
        y=data[class_labels == 1, 1],  # Y-coordinates of Class 1
        z=data[class_labels == 1, 2],  # Z-coordinates of Class 1
        mode='markers',  # Use markers to represent points
        marker=dict(
            size=5,  # Size of the markers
            color='green',  # Color of markers for Class 1
            opacity=0.8  # Marker opacity
        ),
        name='Class 1 (Green)'  # Label for legend
    )
])

# Update the layout to improve plot presentation
fig.update_layout(
    title="Data and True Class Labels",  # Title of the plot
    margin=dict(l=0, r=0, b=0, t=100),  # Adjust plot margins (left, right, bottom, top)
    annotations=[  # Annotations for the plot
        dict(
            text="Class 0: Yellow, Class 1: Green",  # Text content
            showarrow=False,  # Do not show an arrow pointing to the text
            xref="paper",  # Reference x position in terms of the plot's width
            yref="paper",  # Reference y position in terms of the plot's height
            x=0.5,  # X position (center of the plot)
            y=1.05,  # Y position (slightly above the plot)
            xanchor="center",  # Anchor the text at its center
            yanchor="bottom",  # Anchor the text at the bottom
            font=dict(  # Font settings for the text
                size=12,
                color="black"
            )
        )
    ]
)

# Display the plot
fig.show()
fig.write_image("plotly_3d_scatter_plot.png")

In [7]:
def classify_with_erm(data, model_parameters, gamma):
    """
    Classifies data points based on the Expected Risk Minimization (ERM) principle using a Gaussian Mixture Model.

    Parameters:
    data (numpy.ndarray): The dataset to classify, where rows correspond to samples and columns to features.
    model_parameters (dict): Dictionary containing the parameters of the GMM ('mu' for means, 'Sigma' for covariance matrices, and 'priors' for class probabilities).
    gamma (float): Decision threshold computed from class priors and loss values.

    Returns:
    numpy.ndarray: An array of predicted class labels for each data point, where class labels are 0 or 1.
    """
    # Calculate the likelihood of data for each class using the Gaussian PDF
    likelihoods_0 = multivariate_normal.pdf(data, mean=model_parameters['mu'][0], cov=model_parameters['Sigma'][0])
    likelihoods_1 = multivariate_normal.pdf(data, mean=model_parameters['mu'][1], cov=model_parameters['Sigma'][1])

    # Compute likelihood ratios
    likelihood_ratio = likelihoods_1 / likelihoods_0

    # Classify based on the computed gamma
    predicted_classes = likelihood_ratio > gamma
    return predicted_classes.astype(int)

In [8]:
def compute_roc_curve(decision_scores, true_labels):
    """
    Computes the Receiver Operating Characteristic (ROC) curve for given decision scores.

    Parameters:
    decision_scores (numpy.ndarray): Array of scores from the classifier.
    true_labels (numpy.ndarray): Actual class labels.

    Returns:
    dict: Dictionary containing arrays for false positive rates ('fpr') and true positive rates ('tpr').
    list: List of threshold values (gammas) used to compute the ROC curve.
    """
    class_counts = np.array([np.sum(true_labels == 0), np.sum(true_labels == 1)])
    sorted_scores = sorted(decision_scores)
    min_epsilon = np.finfo(float).eps

    thresholds = [sorted_scores[0] - min_epsilon] + sorted_scores + [sorted_scores[-1] + min_epsilon]
    decision_outcomes = [decision_scores >= gamma for gamma in thresholds]

    false_positives = [np.sum((outcome == 1) & (true_labels == 0)) for outcome in decision_outcomes]
    true_positives = [np.sum((outcome == 1) & (true_labels == 1)) for outcome in decision_outcomes]

    false_positive_rate = np.array(false_positives) / class_counts[0]
    true_positive_rate = np.array(true_positives) / class_counts[1]

    roc_data = {
        'fpr': false_positive_rate,
        'tpr': true_positive_rate
    }

    return roc_data, thresholds

In [9]:
def compute_classification_metrics(predicted_labels, actual_labels):
    """
    Computes classification metrics including True Negative Rate (TNR), False Positive Rate (FPR),
    False Negative Rate (FNR), and True Positive Rate (TPR).

    Parameters:
    predicted_labels (numpy.ndarray): Predicted class labels.
    actual_labels (numpy.ndarray): Actual class labels.

    Returns:
    dict: Dictionary containing classification metrics ('TNR', 'FPR', 'FNR', 'TPR').
    """
    class_counts = np.array([np.sum(actual_labels == 0), np.sum(actual_labels == 1)])

    TN = np.sum((predicted_labels == 0) & (actual_labels == 0))
    FP = np.sum((predicted_labels == 1) & (actual_labels == 0))
    FN = np.sum((predicted_labels == 0) & (actual_labels == 1))
    TP = np.sum((predicted_labels == 1) & (actual_labels == 1))

    metrics = {
        'TNR': TN / class_counts[0],
        'FPR': FP / class_counts[0],
        'FNR': FN / class_counts[1],
        'TPR': TP / class_counts[1]
    }

    return metrics

In [10]:
# Assuming gmm_parameters have been defined and used for generating data
conditional_probabilities = np.array([
    multivariate_normal.pdf(data, mean=gmm_parameters['mu'][class_index], cov=gmm_parameters['Sigma'][class_index])
    for class_index in range(len(gmm_parameters['priors']))
])
likelihood_ratio_scores = np.log(conditional_probabilities[1] / conditional_probabilities[0])


roc_data, threshold_values = compute_roc_curve(likelihood_ratio_scores, class_labels)

# Creating a Plotly figure for the ROC curve
roc_curve_figure = go.Figure()

# Adding the ROC curve trace
roc_curve_figure.add_trace(go.Scatter(
    x=roc_data['fpr'],  # False Positive Rate
    y=roc_data['tpr'],  # True Positive Rate
    mode='lines+markers',
    name='Empirical ERM Classifier ROC Curve',
    marker=dict(color='blue'),  # Marker color
    line=dict(dash='dash')  # Line style
))

# Setting the layout for the plot
roc_curve_figure.update_layout(
    title='ROC Curve for ERM Classifier',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    legend_title="Legend",
    template="plotly_white"
)

# Enabling grid lines for better readability
roc_curve_figure.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightPink')
roc_curve_figure.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightBlue')

# Displaying the ROC curve plot
roc_curve_figure.show()
roc_curve_figure.write_image("ROC_plot.png")

In [11]:
# Calculate the error probabilities using the computed ROC data and class distribution
error_probabilities = np.array([roc_data['fpr'], 1 - roc_data['tpr']]).T.dot(class_distribution / total_samples)

# Find the minimum error probability and its index
minimum_error_probability = np.min(error_probabilities)
minimum_error_index = np.argmin(error_probabilities)

# Calculate the theoretical gamma from class priors
theoretical_gamma = gmm_parameters['priors'][0] / gmm_parameters['priors'][1]
# Decision mapping based on the likelihood ratio and theoretical gamma
decision_map = likelihood_ratio_scores >= np.log(theoretical_gamma)

# Calculate classification metrics for the mapped decisions
# def compute_classification_metrics(predicted_labels, actual_labels):
classification_metrics = compute_classification_metrics(decision_map, class_labels)

# Minimum theoretical probability of error
minimum_theoretical_error = np.array([classification_metrics['FPR'] * gmm_parameters['priors'][0] +
                                      classification_metrics['FNR'] * gmm_parameters['priors'][1]])

# Adding markers to the ROC plot to highlight the minimum error probabilities


roc_curve_figure.add_trace(go.Scatter(
    x=[roc_data['fpr'][minimum_error_index]],
    y=[roc_data['tpr'][minimum_error_index]],
    mode='markers',
    name=f"Empirical Min error ERM = {minimum_error_probability:.3f}, Gamma = {np.exp(threshold_values[minimum_error_index]):.3f}",
    marker=dict(color='green', size=20)  # Green marker for empirical minimum error
))

roc_curve_figure.add_trace(go.Scatter(
    x=[classification_metrics['FPR']],
    y=[classification_metrics['TPR']],
    mode='markers',
    name=f"Theoretical Min Pr(error) ERM = {minimum_theoretical_error[0]:.3f}, Gamma = {theoretical_gamma:.3f}",
    marker=dict(color='red', size=14)  # Red marker for theoretical minimum error
))

# Print the minimum error probabilities and corresponding gamma values
print(f"Minimum Empirical error for ERM = {minimum_error_probability:.3f}")
print(f"Minimum Empirical Gamma = {np.exp(threshold_values[minimum_error_index]):.3f}")
print(f"Minimum Theoretical error  for ERM = {minimum_theoretical_error[0]:.3f}")
print(f"Minimum Theoretical Gamma = {theoretical_gamma:.3f}")

# Update the plot layout and display it
roc_curve_figure.update_layout(legend_title="Legend", showlegend=True)
roc_curve_figure.show()
roc_curve_figure.write_image("ROC_plot1.png")

Minimum Empirical error for ERM = 0.084
Minimum Empirical Gamma = 2.318
Minimum Theoretical error  for ERM = 0.084
Minimum Theoretical Gamma = 2.333


In [12]:
# Calculate naive conditional likelihoods assuming independence among features
naive_conditional_likelihoods = np.array([
    multivariate_normal.pdf(data, mean=gmm_parameters['mu'][l], cov=np.eye(dimensionality))
    for l in range(len(gmm_parameters['priors']))
])
# Calculate the likelihood ratio for naive classification
naive_likelihood_ratio = np.log(naive_conditional_likelihoods[1] / naive_conditional_likelihoods[0])

# Compute ROC curve and error probabilities for naive classifier
naive_roc_data, naive_thresholds = compute_roc_curve(naive_likelihood_ratio, class_labels)

naive_error_probabilities = np.array([naive_roc_data['fpr'], 1 - naive_roc_data['tpr']]).T.dot(class_distribution / total_samples)

# Min prob error for the naive classifier gamma thresholds
min_prob_error_naive = np.min(naive_error_probabilities)
min_ind_naive = np.argmin(naive_error_probabilities)
min_naive_gamma = np.exp(naive_thresholds[min_ind_naive])

# Print the minimum probability of error and gamma value for naive classifier
print("Min Naive for ERM = {:.3f}".format(min_prob_error_naive))
print("Min Naive Gamma = {:.3f}".format(np.exp(naive_thresholds[min_ind_naive])))

# Adding the naive ROC curve trace
roc_curve_figure.add_trace(go.Scatter(
    x=naive_roc_data['fpr'],
    y=naive_roc_data['tpr'],
    mode='lines+markers',
    name=f'Naive Empirical ROC - Min error= {min_prob_error_naive:.3f}, Gamma = {min_naive_gamma:.3f}',
    marker=dict(color='purple')  # Purple color for differentiation
))

# Highlight the minimum error probability point for the naive classifier
roc_curve_figure.add_trace(go.Scatter(
    x=[naive_roc_data['fpr'][minimum_error_index]],
    y=[naive_roc_data['tpr'][minimum_error_index]],
    mode='markers+text',  # Adding text to point out the marker
    name=f'Naive Min ERM = {min_prob_error_naive:.3f}, Gamma = {min_naive_gamma:.3f}',
    text=["Min Error"],
    textposition="top center",
    marker=dict(color='red', symbol='cross', size=14)  # Red cross marker
))

# Setting the layout for the plot
roc_curve_figure.update_layout(
    title='ROC Curve for ERM Classifier',
    xaxis_title='False Alarm',
    yaxis_title='True Positive Alarm',
    legend_title="Legend",
    template="plotly_white"
)

# Enabling grid lines for better readability
roc_curve_figure.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightPink')
roc_curve_figure.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightBlue')

# Displaying the ROC curve plot
roc_curve_figure.show()
# Assuming roc_curve_figure is your Plotly figure object
roc_curve_figure.write_image("ROC_plot2.png")

Min Naive for ERM = 0.098
Min Naive Gamma = 3.274
